In [1]:
import pandas as pd
import numpy as np

dft = pd.read_csv("times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

In [3]:
list(set(dft.columns)-set(dft.index))

['Cremona, Ugolani Dati, 4.1',
 'Cremona, Palosca n. 2.1',
 'Ostiano',
 'San Giovanni',
 'Cremona, Palosca n. 2']